# Demo
This notebook provides visualizations of the recovered faces.

In [ ]:
import matplotlib.pyplot as plt
from eval_attack import for_figure

from backbones import get_backbone_v2
from feats.utils_feat import load_bin
from preset import get_LSH_preset
from INV import get_nbnet

device = "" # your device setting "cpu" or "cuda"

### Load LSH preset

In [ ]:
LSHs = get_LSH_preset(device)

### Load dataset & pre-trained models

In [ ]:
test_dir = "" # input your dataset(LFW)'s directory
lfw = load_bin(test_dir + '/lfw.bin', (112,112))

target_dir = "" # input your pre-trained target backbone model's directory
backbone = get_backbone_v2(("r50", target_dir, False), device) # If supported by get_backbone_v2, you can change "r50" to  other models.

nbnet_dir = "" # input your pre-trainde NbNet model's directory
nbnet = get_nbnet(nbnet_dir, device)

### Function for printing figures

##### The first row shows the results of an attack using the genetic method, and the second row shows the results of an attack using the proposed method. 

##### The numbers on each image mean, in order from the top, the lsh score and purity in a type 1 attack, and the lsh score and cosine similarity in a type 2 attack.

**NOTE** This reproduces Figure 2.

In [ ]:
def print_fig(mem_gen, mem_ours, source):
    
                           
    ten2img = lambda x: (x[0].detach().cpu().numpy().transpose(1,2,0) + 1) / 2
    
    n_figures = len(mem_ours) + 1
    
    plt.figure(figsize=(15,15))
    plt.subplot(2, n_figures, 1)
    plt.imshow(ten2img(source))
    plt.axis("off")
    plt.title("Source", fontsize = 10)
    
    for idx, payload in enumerate(mem_gen):
        name, img, score1, score2, purity, cosine = payload
        
        plt.subplot(2, n_figures, idx + 2)
        plt.imshow(ten2img(img))
        plt.axis("off")
        plt.title(name + f"\n{score1}\n{purity:.4f}\n{score2}\n{cosine:.4f}", fontsize = 10)
        
    plt.subplot(2, n_figures, n_figures + 1)
    plt.imshow(ten2img(source))
    plt.axis("off")
    plt.title("Source", fontsize = 10)
    
    for idx, payload in enumerate(mem_ours):
        name, img, score1, score2, purity, cosine = payload
        
        plt.subplot(2, n_figures, n_figures + idx + 2)
        plt.imshow(ten2img(img))
        plt.axis("off")
        plt.title(name + f"\n{score1}\n{purity:.4f}\n{score2}\n{cosine:.4f}", fontsize = 10)

    plt.show()

### Generate figure's info 
##### (attack with proposed method, genetic method about input index of dataset)

##### If there is no input for the index, it proceeds to a random index.

In [ ]:
mem_gen, mem_ours, source = for_figure(lfw, LSHs, backbone, nbnet, device, index = None)
print_fig(mem_gen, mem_ours, source)

### Index set of figure 2.

In [ ]:
idx_set = [1105, 10059, 11737]
for ind in idx_set:
    mem_gen, mem_ours, source = for_figure(lfw, LSHs, backbone, nbnet, device, index = ind)
    print_fig(mem_gen, mem_ours, source)